## 셀레니움을 이용한 카카오맵 크롤링
---
지도 크롤링 하면 대개 네이버 지도를 주로 사용합니다만, 파이썬(Python) 동적 크롤링으로는 카카오맵 또한 크롤링을 할 수 있습니다. 개인적으로는 광고가 검색결과와 분리되어 있는 점, 기본 화면에서 바로 복사할 수 있는 정보가 많다는 점에서 카카오맵을 선호하지만, 네이버 지도에 비해서 등록된 업체의 수가 적다는 점은 단점으로 꼽히니 구하고자 하는 데이터의 특성에 따라 적절하게 고르는게 좋겠습니다.  
</p></br></br>


카카오맵 크롤링 또한 셀레니움(selenium)이라고 하는 파이썬 패키지를 이용해서 진행합니다. 특정 키워드를 중심으로 지도 페이지를 검색한 다음, 검색 결과의 모든 정보를 복사하는 과정을 수행합니다. 이렇게 해당 페이지의 모든 정보를 복사했다면 더이상 불가능할 때까지 다음 페이지에서 동일한 작업을 진행하는 방식으로 동작하도록 설계했습니다.  
</p></br></br>


동작 코드는 아래 코드를 참고해 보시면 되겠습니다. 저는 서울특별시 종로구의 어학원을 모두 리스트업하는 상황을 상정해서 코드를 작성해 봤습니다.  
</p></br></br>



In [1]:
# 패키지 불러오기
from selenium import webdriver
import pandas as pd

driver = webdriver.Chrome()

# 검색어 입력 및 검색
keyword = '서울 종로구 어학원'
kakao_map_search_url = f"https://map.kakao.com/?q={keyword}"

driver.get(kakao_map_search_url)
driver.get(kakao_map_search_url)

</p></br></br>


카카오맵에서는 업체명과 주소가 각각 `//*[@id="info.search.place.list"]/li[인덱스]/div[3]/strong/a[2]` 와 `//*[@id="info.search.place.list"]/li[인덱스]/div[5]/div[2]/p[1]` 라는 XPATH로 지정되어 있습니다. 그래서, 각각의 반복마다 인덱스 숫자를 늘려가면서 몇번째 업체명과 주소인지를 확인하고 복사하도록 반복문을 구축해 봤어요.  
</p></br></br>


그리고, 카카오맵은 첫 페이지에 페이지 이동 버튼이 보이지 않는다는 특징이 있습니다. 1페이지에서는 '장소 더보기' 라는 버튼을 눌러야 2페이지로 이동하고, 2페이지부터 특정 페이지로 이동할 수 있도록 구성되어 있습니다. 그래서, 더보기 버튼(XPATH 기준 `//*[@id="info.search.place.more"]` )이 있다면 1페이지, 아니면 2페이지 이상으로 간주하고 해당 페이지에 맞는 동작을 취하도록 코드를 구성했습니다.  
</p></br></br>



In [2]:
ind = 1  # 현재 복사한 순서
no = 1  # 1~5페이지 중 위치한 곳
page = 1  # 현재 페이지 번호
list1 = []  # 결과물이 저장되는 리스트

while 1:
    try:
        # 업체명, 주소
        title = driver.find_element(by='xpath',value = f'//*[@id="info.search.place.list"]/li[{ind}]/div[3]/strong/a[2]').text
        addr = driver.find_element(by='xpath',value = f'//*[@id="info.search.place.list"]/li[{ind}]/div[5]/div[2]/p[1]').text
        list1.append([title, addr])
        
        ind += 1
        
    except:
        # 더보기 버튼 찾기
        if driver.find_element(by='xpath',value = f'//*[@id="info.search.place.more"]').is_displayed():
            driver.find_element(by='xpath',value = f'//*[@id="info.search.place.more"]').click()
            no += 1
            ind = 1
            page += 1
            continue

        # 다음 페이지로 이동
        elif no >= 5:
            driver.find_element(by='xpath',value = f'//*[@id="info.search.page.next"]').click()
            no = 1
            ind = 1
            page += 1
            continue
            
        # 5페이지 단위마다 다음 페이지 버튼 누르기
        elif driver.find_element(by='xpath',value = f'//*[@id="info.search.page.no{no+1}"]').is_displayed():
            no += 1
            driver.find_element(by='xpath',value = f'//*[@id="info.search.page.no{no}"]').click()
            ind = 1
            page += 1
            continue
        
        # 더이상 이동할수 없을 경우 종료
        else:
            break

</p></br></br>


위 코드로 업체 정보를 크롤링한 결과는 2차원 리스트의 형태로 저장되는데, 이는 파이썬에서 판다스 데이터프레임(Pandas DataFrame)으로 변환한다면 우리가 흔히 쓰는 표 형태로 이용할 수 있습니다. 이 다음에는 `to_excel` 함수 또는 `to_csv` 함수 등을 이용해서 xlsx 또는 csv 파일로 내보낼수도 있지요.  
</p></br></br>



In [3]:
pd.DataFrame(list1)

,0,1
0,YBM어학원 종로센터,서울 종로구 종로 104
1,랭귀지큐브 종로센터,서울 종로구 삼봉로 57 호수빌딩 6층
2,이치고이치에학원,서울 종로구 삼일대로15길 17 금옥학술재안빌딩 8층
3,시사아카데미법인,서울 종로구 삼일대로17길 16
4,플랜티어학원,"서울 종로구 수표로 105 육의전빌딩 4층,8~9층"
...,...,...
257,초이스잉글리쉬학원,서울 강북구 인수봉로 10
258,서비스교육센터어학원,서울 중구 동호로20가길 16
259,참길영어수학학원,서울 성북구 삼양로 74
260,제이제이영어,서울 은평구 녹번로 7-9
